# Merge tokenized gradients to word-level for easier comparisons

In [2]:
import pandas as pd
from datasets import load_from_disk
from gradient_utils import *
from model_utils import *
import os

## Experimental Data

In [79]:
df = pd.read_csv(f"High_Precision.tsv", sep="\t")

out = []

for i,row in df.iterrows():
    MODEL = row['model']
    data = row['data']
    PATTERN = row['pattenr']
    pt = row['pt']
    nk = row['nk']
    tt = None
    if 'token' in row['nk']:
        tt = row['nk'].split('-')[1]
        nk = 'token'
    if not pd.isna(row['lvl']):
        lvl = f"{int(row['lvl']):02d}"
    else:
        lvl = np.nan
    k = row['grad']
    idx = row['id']
    stats = [MODEL,data,PATTERN,pt,nk,lvl,k,idx]
    
    if PATTERN == 'human':
        if pt == 'A':
            text_label = f"{PATTERN}-{pt}_{nk}"
        else:
            text_label = f"{PATTERN}-{pt}_{nk}_{lvl}"
    elif PATTERN == 'gradient':
        text_label = f"{PATTERN}-{MODEL}_{nk}_{lvl}"
    else: # random
        text_label = f"{PATTERN}_{nk}_{lvl}"
        
    for i, val in enumerate(load_from_disk(f"./Data/Noise/{data}")['index']):
        if val == idx:
            break
    orig_data =  load_from_disk(f"./Data/Noise/{data}")[text_label][i]
    
    word2attributions, pred_token_dic, new_data_record = process_bert(pd.DataFrame(row).transpose(), [orig_data], MODEL, tt)
    out.append(new_data_record + stats)

clean_out = pd.DataFrame(out)

KeyboardInterrupt: 

In [ ]:
clean_out = pd.DataFrame(out, columns = [ 'vis', 'ids', 'entropies', 'model', 'data', 'pattern', 'pt', 'nk', 'lvl', 'k', 'i'])

filename = "High_Precision_CLEAN"
preds_ids,label_ids = [], []
tsv_file = filename+".tsv"
tsv_rows = []
for i,row in clean_out.iterrows():
    datarecord = row['vis']
    prediction = str(int(datarecord.pred_class))
    truth = str(int(datarecord.true_class))
    prediction_prob = "{:.2f}".format(float(datarecord.pred_prob))
    the_raw_input = datarecord.raw_input_ids
    word_attr_scores = datarecord.word_attributions
    temp = []
    importance_list = []
    for word, importance in zip(the_raw_input, word_attr_scores[:len(the_raw_input)]):
        word = format_special_tokens(word)
        temp_str = word #+ " ({:.2f})".format(importance)
        temp.append(temp_str)
        importance_list.append("{:.6f}".format(importance))
    text = " ".join(temp)
    attr_list = " ".join(importance_list)
    tsv_row = [str(row['ids']), prediction, truth, prediction_prob, str(row['entropies']), text, attr_list] 
    tsv_row += [row['model'], row['data'], row['pattern'], row['pt'], row['nk'], row['lvl'], row['k'], str(row['i'])]
    tsv_rows.append(tsv_row)

    preds_ids.append(int(datarecord.pred_class))
    label_ids.append(int(datarecord.true_class))
    
    with open(tsv_file, 'w') as writer:
        header = "id\tpred_class\ttrue_class\tpred_prob\tpred_entropy\traw_input\tattribution_scores\tmodel\tdata\tpattern\tpt\tnk\tlvl\tgrad\tindex\n"
        writer.write(header)
        for row in tsv_rows:
            to_be_written = "\t".join(row)
            writer.write(to_be_written+"\n")


print("Completed.")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0,0.86,0.484509,"[besideJs, i, sam, irish, xwhite, niggeir, Esp...","[0.0008160674157303363, 0.001323681818181821, ...",bert,hatexplain,gradient,NaN,charinsert,70,GuidedBP,0
1,1,2,0.47,1.045175,"[bQest, live, tRopic, iP, lhave, seeHn, on, gu...","[-0.023614117647058824, -0.02321152, -0.022234...",bert,hatexplain,gradient,NaN,charinsert,70,GuidedBP,1
2,1,2,0.69,0.779557,"[ymou, aYre, obgviously, stPupid, Vso, your, w...","[0.030288333333333334, 0.005642023255813951, 0...",bert,hatexplain,gradient,NaN,charinsert,70,GuidedBP,2
3,0,0,0.94,0.254187,"[whiteG, peoplSe, whlo, havHe, enouYgh, rDacia...","[-0.004366865853658536, -0.005146246913580247,...",bert,hatexplain,gradient,NaN,charinsert,70,GuidedBP,3
4,1,2,0.52,0.729957,"[totalbiscuitM, mainvly, attacBked, sjwAs, bec...","[-0.004618447368421051, -0.006321787610619465,...",bert,hatexplain,gradient,NaN,charinsert,70,GuidedBP,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,1,2,0.74,0.645293,"[theR, Nre, callinx, nick, fuentesW, kot, paJt...","[0.00262111, 0.002733414141414141, 0.002551857...",bert,hatexplain,gradient,NaN,charswap,90,InputXGrad,76
77,1,2,0.76,0.595876,"[Tt, usahotJips, q, nevwr, thoufht, i, wvuld, ...","[-0.0023056292134831452, -0.001792874999999998...",bert,hatexplain,gradient,NaN,charswap,90,IntegGrad,77
78,1,2,0.85,0.454581,"[notL, onGy, thak, Dut, thesej, peopleU, fleQ,...","[-0.0022923717948717904, -0.002935220779220778...",bert,hatexplain,gradient,NaN,charswap,90,IntegGrad,78
79,1,2,0.86,0.438401,"[comeC, onR, pleast, yous, forgotP, youV, arl,...","[-0.018766569444444443, -0.018841647887323947,...",bert,hatexplain,gradient,NaN,charswap,95,GuidedBP,79


In [ ]:
bert_data = pd.read_csv(f"Saliency/RAW/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}.tsv", sep="\t") # dataframe
                        # orig_data = load_from_disk(f"./Data/Noise/{test_data}")[text_label] # text form
                        
                        orig_data_unsorted = pd.DataFrame(load_from_disk(f"./Data/Noise/{test_data}")[text_label], columns=['text'])
                        orig_data_unsorted.index = load_from_disk(f"./Data/Noise/{test_data}")['index']
                        orig_data = orig_data_unsorted.loc[bert_data['id'].tolist()]['text']

                        word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model, token_type)

                        directory = f"Saliency/Clean/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}/"
                        if not os.path.exists(directory):
                        # If it doesn't exist, create it
                            os.makedirs(directory)
                            
                        visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{k}")

In [ ]:
a

In [6]:
PATTERN = 'human'
model = 'bert'
dataset = 'SST-2'
pt = 'S'
nk = 'synonym'
lvl = '90'
k = 'SmoothGrad'

bert_data = pd.read_csv(f"Saliency/RAW/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}.tsv", sep="\t", index_col=0) # dataframe

bert_data = pd.read_csv(f"Saliency/RAW/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}.tsv", sep="\t") # dataframe
# orig_data = load_from_disk(f"./Data/Noise/{test_data}")[text_label] # text form

orig_data_unsorted = pd.DataFrame(load_from_disk(f"./Data/Noise/{dataset}"), columns=['text'])
orig_data_unsorted.index = load_from_disk(f"./Data/Noise/{dataset}")['index']
orig_data = orig_data_unsorted.loc[bert_data['id'].tolist()]['text']

word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model, None)

directory = f"Saliency/Clean/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}/"
if not os.path.exists(directory):
# If it doesn't exist, create it
    os.makedirs(directory)
    
new_data_record

[{'pred_class': 0.0,
  'true_class': 1.0,
  'pred_prob': 1.0,
  'pred_entropy': 0.006981293,
  'raw_input_ids': ['an',
   'artful',
   'intelligent',
   'film',
   'that',
   'stays',
   'within',
   'the',
   'confines',
   'of',
   'a',
   'well-established',
   'genre'],
  'word_attributions': [0.2,
   0.15625,
   0.14956521739130432,
   0.14681818181818182,
   0.14285714285714285,
   0.1335,
   0.12736842105263158,
   0.12388888888888888,
   0.12352941176470589,
   0.12375,
   0.12533333333333332,
   0.12642857142857142,
   0.12615384615384617]},
 {'pred_class': 1.0,
  'true_class': 1.0,
  'pred_prob': 1.0,
  'pred_entropy': 0.0022327283,
  'raw_input_ids': ['an',
   'exhilarating',
   'futuristic',
   'thriller-noir',
   'minority',
   'report',
   'twists',
   'the',
   'best',
   'of',
   'technology',
   'around',
   'a',
   'gripping',
   'story',
   'delivering',
   'a',
   'riveting',
   'pulse',
   'intensifying',
   'escapist',
   'adventure',
   'of',
   'the',
   'first'

In [2]:
pts = ['R'] #['S', 'R']
nks = ['token-unk','token-mask', 'charswap', 'synonym', 'butterfingers', 'wordswap', 'charinsert', 'l33t']
# lvls = ['25', '50', '70', '80', '90', '95']
models = ['opt'] #['bert', 'electra', 'roberta'] 
datasets = ['SST-2']
lvls = ['05', '10', '25', '50', '70', '80', '90', '95']

In [6]:
PATTERN = 'human'

for model in models:
    for dataset in datasets:
        for pt in pts:
            for nk in nks: # kinds of noise
                for lvl in lvls: # levels of noise
        
                    if "token" in nk:
                        text_label = f"{PATTERN}-{pt}_{nk.split('-')[0]}_{lvl}"
                        token_type = nk.split('-')[1]
                    else:
                        text_label = f"{PATTERN}-{pt}_{nk}_{lvl}" # human-R_token_{prop*100:2.0f}'
                        token_type = None
                    test_data = dataset

                    ## Get Data

                    for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
                        bert_data = pd.read_csv(f"Saliency/RAW/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}.tsv", sep="\t") # dataframe
                        # orig_data = load_from_disk(f"./Data/Noise/{test_data}")[text_label] # text form
                        
                        orig_data_unsorted = pd.DataFrame(load_from_disk(f"./Data/Noise/{test_data}")[text_label], columns=['text'])
                        orig_data_unsorted.index = load_from_disk(f"./Data/Noise/{test_data}")['index']
                        orig_data = orig_data_unsorted.loc[bert_data['id'].tolist()]['text']

                        word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model, token_type)

                        directory = f"Saliency/Clean/{model}/{PATTERN}/{pt}/{nk}/{lvl}/{dataset}/{k}/"
                        if not os.path.exists(directory):
                        # If it doesn't exist, create it
                            os.makedirs(directory)
                            
                        visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{k}")


visualize prediction text to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/GuidedBP/GuidedBP
done
finish writing to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/GuidedBP/GuidedBP
visualize prediction text to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/InputXGrad/InputXGrad
done
finish writing to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/InputXGrad/InputXGrad
visualize prediction text to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/IntegGrad/IntegGrad
done
finish writing to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/IntegGrad/IntegGrad
visualize prediction text to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/SmoothGrad/SmoothGrad
done
finish writing to  Saliency/Clean/bert/human/R/token-unk/25/SemEval/SmoothGrad/SmoothGrad
visualize prediction text to  Saliency/Clean/bert/human/R/token-unk/50/SemEval/GuidedBP/GuidedBP
done
finish writing to  Saliency/Clean/bert/human/R/token-unk/50/SemEval/GuidedBP/GuidedBP
visualize prediction text to  Saliency/C

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
PATTERN = 'human'

for model in models:
    for dataset in datasets:
        pt = 'A'
        for nk in nks: # kinds of noise
            
            if "token" in nk:
                text_label = f"{PATTERN}-{pt}_{nk.split('-')[0]}"
                token_type = nk.split('-')[1]
            else:
                text_label = f"{PATTERN}-{pt}_{nk}" # human-R_token_{prop*100:2.0f}'
                token_type = None
                        
            # text_label = f"{PATTERN}-{pt}_{nk.split('-')[0]}"
            test_data = dataset

            ## Get Data

            for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
                bert_data = pd.read_csv(f"Saliency/RAW/{model}/{PATTERN}/{pt}/{nk}/{dataset}/{k}.tsv", sep="\t") # dataframe
                # orig_data = load_from_disk(f"./Data/Noise/{test_data}")[text_label] # text form
                
                orig_data_unsorted = pd.DataFrame(load_from_disk(f"./Data/Noise/{test_data}")[text_label], columns=['text'])
                orig_data_unsorted.index = load_from_disk(f"./Data/Noise/{test_data}")['index']
                orig_data = orig_data_unsorted.loc[bert_data['id'].tolist()]['text']
                        
                word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model, token_type)


                directory = f"Saliency/Clean/{model}/{PATTERN}/{pt}/{nk}/{dataset}/{k}/"
                if not os.path.exists(directory):
                # If it doesn't exist, create it
                    os.makedirs(directory)
                    
                visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{k}")


visualize prediction text to  Saliency/Clean/bert/human/A/token-mask/SemEval/GuidedBP/GuidedBP
done
finish writing to  Saliency/Clean/bert/human/A/token-mask/SemEval/GuidedBP/GuidedBP
visualize prediction text to  Saliency/Clean/bert/human/A/token-mask/SemEval/InputXGrad/InputXGrad
done
finish writing to  Saliency/Clean/bert/human/A/token-mask/SemEval/InputXGrad/InputXGrad
visualize prediction text to  Saliency/Clean/bert/human/A/token-mask/SemEval/IntegGrad/IntegGrad
done
finish writing to  Saliency/Clean/bert/human/A/token-mask/SemEval/IntegGrad/IntegGrad
visualize prediction text to  Saliency/Clean/bert/human/A/token-mask/SemEval/SmoothGrad/SmoothGrad
done
finish writing to  Saliency/Clean/bert/human/A/token-mask/SemEval/SmoothGrad/SmoothGrad
visualize prediction text to  Saliency/Clean/bert/human/A/token-unk/SemEval/GuidedBP/GuidedBP
done
finish writing to  Saliency/Clean/bert/human/A/token-unk/SemEval/GuidedBP/GuidedBP
visualize prediction text to  Saliency/Clean/bert/human/A/toke

In [ ]:
PATTERN = 'random'

for model in models:
    for dataset in datasets:
        for nk in nks: # kinds of noise
            for lvl in lvls: # levels of noise
        
                if "token" in nk:
                    text_label = f"{PATTERN}_{nk.split('-')[0]}_{lvl}"
                    token_type = nk.split('-')[1]
                else:
                    text_label = f"{PATTERN}_{nk}_{lvl}" # human-R_token_{prop*100:2.0f}'
                    token_type = None
                test_data = dataset

                ## Get Data

                for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
                    bert_data = pd.read_csv(f"Saliency/RAW/{model}/{PATTERN}/{nk}/{lvl}/{dataset}/{k}.tsv", sep="\t") # dataframe
                    # orig_data = load_from_disk(f"./Data/Noise/{test_data}")[text_label] # text form
                    orig_data_unsorted = pd.DataFrame(load_from_disk(f"./Data/Noise/{test_data}")[text_label], columns=['text'])
                    orig_data_unsorted.index = load_from_disk(f"./Data/Noise/{test_data}")['index']
                    orig_data = orig_data_unsorted.loc[bert_data['id'].tolist()]['text']
                    
                    word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model, token_type)

                    directory = f"Saliency/Clean/{model}/{PATTERN}/{nk}/{lvl}/{dataset}/{k}/"
                    if not os.path.exists(directory):
                    # If it doesn't exist, create it
                        os.makedirs(directory)
                        
                    visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{k}")


visualize prediction text to  Saliency/Clean/bert/random/token-mask/25/SemEval/GuidedBP/GuidedBP
done
finish writing to  Saliency/Clean/bert/random/token-mask/25/SemEval/GuidedBP/GuidedBP
visualize prediction text to  Saliency/Clean/bert/random/token-mask/25/SemEval/InputXGrad/InputXGrad
done
finish writing to  Saliency/Clean/bert/random/token-mask/25/SemEval/InputXGrad/InputXGrad
visualize prediction text to  Saliency/Clean/bert/random/token-mask/25/SemEval/IntegGrad/IntegGrad
done
finish writing to  Saliency/Clean/bert/random/token-mask/25/SemEval/IntegGrad/IntegGrad
visualize prediction text to  Saliency/Clean/bert/random/token-mask/25/SemEval/SmoothGrad/SmoothGrad
done
finish writing to  Saliency/Clean/bert/random/token-mask/25/SemEval/SmoothGrad/SmoothGrad
visualize prediction text to  Saliency/Clean/bert/random/token-mask/50/SemEval/GuidedBP/GuidedBP
done
finish writing to  Saliency/Clean/bert/random/token-mask/50/SemEval/GuidedBP/GuidedBP
visualize prediction text to  Saliency/C

In [6]:
prefix = "/tmp/svm-erda/"
PATTERN = 'gradient'
noise_folder = "Noise"
datasets = [ 'esnli']
models = [ 'roberta']
for model in models:
    for dataset in datasets:
        for nk in nks: # kinds of noise
            for lvl in lvls: # levels of noise
    
                if "token" in nk:
                    text_label = f"{PATTERN}-{model}_{nk.split('-')[0]}_{lvl}"
                    token_type = nk.split('-')[1]
                else:
                    text_label = f"{PATTERN}-{model}_{nk}_{lvl}" 
                    token_type = None
                test_data = dataset

                ## Get Data

                for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
                    bert_data = pd.read_csv(f"{prefix}Saliency/RAW/{model}/{PATTERN}/{nk}/{lvl}/{dataset}/{k}.tsv", sep="\t") # dataframe
                    # orig_data = load_from_disk(f"./Data/Noise/{test_data}")[text_label] # text form
                    
                    orig_data_unsorted = pd.DataFrame(load_from_disk(f"./Data/{noise_folder}/{test_data}")[text_label], columns=['text'])
                    orig_data_unsorted.index = load_from_disk(f"./Data/{noise_folder}/{test_data}")['index']
                    orig_data = orig_data_unsorted.loc[bert_data['id'].tolist()]['text']
                
                    word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model, token_type)

                    directory = f"{prefix}Saliency/Clean/{model}/{PATTERN}/{nk}/{lvl}/{dataset}/{k}/"
                    if not os.path.exists(directory):
                    # If it doesn't exist, create it
                        os.makedirs(directory)
                        
                    visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{k}")


visualize prediction text to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/GuidedBP/GuidedBP
done
finish writing to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/GuidedBP/GuidedBP
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/InputXGrad/InputXGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/InputXGrad/InputXGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/IntegGrad/IntegGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/IntegGrad/IntegGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/SmoothGrad/SmoothGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/05/esnli/SmoothGrad/SmoothGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/roberta/gradient/token-unk/10/esnli/Guide

# For Clean Data

In [10]:
pd.DataFrame(zip(load_from_disk(f"./Data/Clean/{test_data}")['test']['text_1'], load_from_disk(f"./Data/Clean/{test_data}")['test']['text_1']))

,0,1
0,This church choir sings to the masses as they ...,This church choir sings to the masses as they ...
1,This church choir sings to the masses as they ...,This church choir sings to the masses as they ...
2,This church choir sings to the masses as they ...,This church choir sings to the masses as they ...
3,"A woman with a green headscarf, blue shirt and...","A woman with a green headscarf, blue shirt and..."
4,"A woman with a green headscarf, blue shirt and...","A woman with a green headscarf, blue shirt and..."
...,...,...
9819,Two women are observing something together.,Two women are observing something together.
9820,Two women are observing something together.,Two women are observing something together.
9821,A man in a black leather jacket and a book in ...,A man in a black leather jacket and a book in ...
9822,A man in a black leather jacket and a book in ...,A man in a black leather jacket and a book in ...


In [3]:
prefix = "/tmp/svm-erda/"
for model in ['opt']: #['bert', 'electra', 'roberta', 'gpt2-medium']:
# for model in ['gpt2-medium']:
    for dataset in  ['SemEval', 'hatexplain']: #[ "esnli"]:
        if dataset == 'SST-2':
            text_label = "sentence"
            test_data = dataset
        elif dataset == 'esnli':
            text_label = ['text_1', 'text_2']
            test_data = dataset

        else:
            text_label = "text"
            test_data = dataset
    # dataset = 'SST-2'
    # text_label = "text"
    # test_data = dataset

        ## Get Data

        for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
            bert_data = pd.read_csv(f"{prefix}Saliency/RAW/{model}/{dataset}-{k}.tsv", sep="\t") # dataframe # pd.read_csv(f"Saliency/Noise0/RAW/{model}/{dataset}-{k}.tsv", sep="\t") # dataframe
            # orig_data = load_from_disk(f"./Data/Clean/{test_data}")['test'][text_label] # text form
            
            if type(text_label) == list:
                test = pd.DataFrame(zip(load_from_disk(f"./Data/Clean/{test_data}")['test']['text_1'], load_from_disk(f"./Data/Clean/{test_data}")['test']['text_2']), columns=text_label)
                test['text'] = test['text_1'] + ' | ' + test['text_2']
                test.index = load_from_disk(f"./Data/Clean/{test_data}")['test']['index']
                orig_data = test.loc[bert_data['id'].tolist()]['text']
            else:
                test = pd.DataFrame(load_from_disk(f"./Data/Clean/{test_data}")['test']['text'], columns=['text'])
                test.index = load_from_disk(f"./Data/Clean/{test_data}")['test']['index']
                orig_data = test.loc[bert_data['id'].tolist()]['text']


            word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model)

            directory = f"{prefix}Saliency/Clean/{model}/"
            if not os.path.exists(directory):
            # If it doesn't exist, create it
                os.makedirs(directory)
                
            visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{dataset}-{k}")


visualize prediction text to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-GuidedBP
done
finish writing to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-GuidedBP
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-InputXGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-InputXGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-IntegGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-IntegGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-SmoothGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/opt/SemEval-SmoothGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/opt/hatexplain-GuidedBP
done
finish writing to  /tmp/svm-erda/Saliency/Clean/opt/hatexplain-GuidedBP
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/opt/hatexplain-InputXGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/opt/hatexplain-InputXGrad
visualize prediction text to  /tmp/svm-erda/Salien

In [3]:
prefix = "/tmp/svm-erda/"
# 
for model in ['bert', 'electra', 'roberta', 'gpt2-medium']:
# for model in ['gpt2-medium']:
    for dataset in [ 'SemEval', 'hatexplain', 'SST-2']:
        if dataset == 'SST-2':
            text_label = "sentence"
            test_data = dataset

        else:
            text_label = "text"
            test_data = dataset
    # dataset = 'SST-2'
    # text_label = "text"
    # test_data = dataset

        ## Get Data
        for c in range(2):

            for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
                bert_data = pd.read_csv(f"/tmp/svm-erda/Saliency/RAW/{model}/Classes/{dataset}/{c}/{k}.tsv", sep="\t") # dataframe # pd.read_csv(f"Saliency/Noise0/RAW/{model}/{dataset}-{k}.tsv", sep="\t") # dataframe
                # orig_data = load_from_disk(f"./Data/Clean/{test_data}")['test'][text_label] # text form
                
                test = pd.DataFrame(load_from_disk(f"./Data/Clean/{test_data}")['test']['text'], columns=['text'])
                test.index = load_from_disk(f"./Data/Clean/{test_data}")['test']['index']
                orig_data = test.loc[bert_data['id'].tolist()]['text']


                word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model)

                directory = f"/tmp/svm-erda/Saliency/Clean/{model}/Classes/{dataset}/{c}/"
                if not os.path.exists(directory):
                # If it doesn't exist, create it
                    os.makedirs(directory)
                    
                visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{k}")


visualize prediction text to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/GuidedBP
done
finish writing to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/GuidedBP
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/InputXGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/InputXGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/IntegGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/IntegGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/SmoothGrad
done
finish writing to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/0/SmoothGrad
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/1/GuidedBP
done
finish writing to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/1/GuidedBP
visualize prediction text to  /tmp/svm-erda/Saliency/Clean/bert/Classes/SemEval/1/InputXGrad
done
finish writi

### Debugging below

In [3]:
def process_bert(bert_data, orig_data, model='bert', token_type=None):

    new_data_record = list(bert_data[['pred_class', 'true_class', 'pred_prob', 'pred_entropy']].transpose().to_dict().values())

    bert_sentences = bert_data["raw_input"].values
    bert_attribution_scores = bert_data["attribution_scores"].values
    pred_probs = bert_data["pred_prob"].values
    
    if token_type:
        g = {'TOKEN': f'[{token_type.upper()}]'}

    word2attributions = defaultdict(list)
    pred_token_dic = defaultdict(list)

    counter = 0
    for sentence, score, orig_text, prob in zip(bert_sentences, bert_attribution_scores, orig_data, pred_probs):
        if token_type:
            orig_text = orig_text.format(**g) ## replace token filler with token used
        bert_tokens, attr_list = remove_cls_and_bert(sentence, score)
        orig2bert, idx2token = match_bert_token_to_original(bert_tokens, orig_text, model)
        new_data_record[counter]['raw_input_ids'] = list(idx2token.values())
        new_attr_dict = make_new_attr_score(orig2bert, attr_list)
        new_data_record[counter]['word_attributions'] = list(new_attr_dict.values())
        for idx, score in new_attr_dict.items():
            real_word = idx2token[idx].replace(",", "").replace("?", "").replace(".", "").replace("!", "").replace("?", "").replace(";", "").replace('"', "").strip()
            word2attributions[real_word].append(score)
            pred_token_dic[real_word].append(prob)

        counter += 1

    return word2attributions, pred_token_dic, [dotdict(x) for x in new_data_record]


def match_bert_token_to_original(bert_tokens, orig_text, model='bert'):
    raw_orig = orig_text.split(" ")
    orig = []
    for word in raw_orig:
        if word != "":
            orig.append(word.strip())
    bert_idx = 0
    last_bert_idx = 1
    orig_idx = 0
    
    orig_to_bert_mapping = {}
    orig_idx2token = {}

    if model in ['bert', 'electra']: #an art ##ful --> An artful
        while bert_idx < len(bert_tokens) and orig_idx < len(orig):

            current_orig = orig[orig_idx]
            current_bert = bert_tokens[bert_idx]
            orig_to_bert_mapping[orig_idx] = [bert_idx]
            
            bert_idx += 1
            orig_idx2token[orig_idx] = current_orig
            if current_bert != current_orig:			
                combined = current_bert
                last_bert_idx = bert_idx
                while last_bert_idx < len(bert_tokens):
                    next_part = bert_tokens[last_bert_idx].replace("##","").strip()				
                    combined += next_part
                    orig_to_bert_mapping[orig_idx].append(last_bert_idx)
                    # if current_orig == "well-established":
                    # 	print("combined: ", combined)
                    if combined == current_orig:					
                        bert_idx = last_bert_idx + 1
                        break
                    else:
                        last_bert_idx += 1

            orig_idx += 1
    else: # roberta  #Ġan Ġart ful --> An artful
        while bert_idx < len(bert_tokens) and orig_idx < len(orig):

            current_orig = orig[orig_idx]
            current_bert = bert_tokens[bert_idx].replace("Ġ","").strip() # Remove any Ġ at the start of word
            orig_to_bert_mapping[orig_idx] = [bert_idx]
            
            bert_idx += 1
            orig_idx2token[orig_idx] = current_orig
            if current_bert != current_orig:			
                combined = current_bert
                last_bert_idx = bert_idx
                while last_bert_idx < len(bert_tokens):
                    next_part = bert_tokens[last_bert_idx]			
                    combined += next_part
                    orig_to_bert_mapping[orig_idx].append(last_bert_idx)
                    # if current_orig == "well-established":
                    # 	print("combined: ", combined)
                    if combined == current_orig:					
                        bert_idx = last_bert_idx + 1
                        break
                    else:
                        last_bert_idx += 1

            orig_idx += 1
    return orig_to_bert_mapping, orig_idx2token



In [30]:
model = "bert"
dataset = "SemEval"
            
text_label = "text"
test_data = dataset
k = "GuidedBP" #for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:

bert_data = pd.read_csv(f"Saliency/Noise0/RAW/{model}/{dataset}-{k}.tsv", sep="\t") # dataframe
# orig_data = load_from_disk(f"./Data/Clean/{test_data}")['test'][text_label] # text form
bert_data

# test = pd.DataFrame(load_from_disk(f"./Data/Clean/{test_data}")['test']['text'], columns=['text'])
# test.index = load_from_disk(f"./Data/Clean/{test_data}")['test']['index']
# orig_data = test.loc[bert_data['id'].tolist()]['text']

# word2attributions, pred_token_dic, new_data_record = process_bert(bert_data, orig_data, model)

# # directory = f"Saliency/Clean/{model}/"
# # if not os.path.exists(directory):
# # # If it doesn't exist, create it
# #     os.makedirs(directory)
    
# # visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"{directory}{dataset}-{k}")

,id,pred_class,true_class,pred_prob,pred_entropy,raw_input,attribution_scores
0,155285232084467712,0,0,1.00,0.012701,[CLS] I just hope Ian Bennett is first out of ...,0.09 0.26 0.05 -0.05 -0.23 -0.09 0.09 0.29 0.1...
1,158551378271277058,0,0,1.00,0.015283,[CLS] Watch ##ed a movie yesterday # The ##70 ...,-0.27 0.09 -0.05 -0.06 0.33 0.38 -0.02 0.05 -0...
2,159200989268807680,0,0,1.00,0.008743,[CLS] I liked a @ YouTube video http : / / t ....,-0.19 -0.10 -0.63 0.31 -0.17 0.19 -0.24 0.21 -...
3,160323475427430400,0,0,0.73,0.584572,[CLS] A win for # L ##FC v Bolton will see the...,-0.21 0.06 0.29 -0.07 0.00 -0.02 0.11 -0.10 -0...
4,160660491574837248,1,1,0.99,0.070319,[CLS] @ co ##urier ##mail I may not agree with...,-0.06 0.14 0.06 0.04 0.20 -0.01 -0.08 0.00 -0....
...,...,...,...,...,...,...,...
1654,264248465440272384,1,1,0.99,0.045548,[CLS] When your lying in bed and your phone is...,-0.14 0.11 0.07 0.18 -0.03 0.16 -0.04 0.02 0.0...
1655,264252575879151616,1,0,0.95,0.197404,[CLS] Arsenal reportedly tracking the progress...,0.18 -0.18 -0.41 0.14 -0.18 0.45 0.14 -0.06 0....
1656,264257216004120576,1,1,0.99,0.039263,[CLS] @ La ##F ##an ##M ##ike ##7 I ' d ask my...,0.02 0.07 -0.01 -0.02 -0.02 -0.03 -0.05 0.02 0...
1657,264258491529719808,0,0,0.99,0.058528,[CLS] The Spurs beat the Thunder 86 - 84 to wi...,-0.45 -0.09 0.22 0.02 0.06 0.03 0.01 0.15 -0.0...


In [24]:
bert_data['id'].tolist()

[155285232084467712,
 158551378271277058,
 159200989268807680,
 160323475427430400,
 162895027431948289,
 165009461449207809,
 168160079529525248,
 169422963077169153,
 169518982519459841,
 169558701781430272,
 169609494815457283,
 169871220244885504,
 170243777422827520,
 171385775416553472,
 172503126190993409,
 172976938464444416,
 173976647131856897,
 173976882000314368,
 173992535138635777,
 174003654238547968,
 175064595055128577,
 175080776684146688,
 176609365422387200,
 177249771285774337,
 177382644273778692,
 177519407940124673,
 177566983741648896,
 177640329724444673,
 177672944506503168,
 177677721940406272,
 177715743314542592,
 177924741301149696,
 178000146481692672,
 178100186453786625,
 178110692853952512,
 178145657150898176,
 178271037396492288,
 178301152318857217,
 178367147955863552,
 178425868341870592,
 178776453427630080,
 179021245587787777,
 179043961955041280,
 179167439861452800,
 179469502935875584,
 179590186261422080,
 179657043358130176,
 179665847445

In [27]:
test = pd.DataFrame(load_from_disk(f"./Data/Clean/{test_data}")['test']['text'], columns=['text'])
test.index = load_from_disk(f"./Data/Clean/{test_data}")['test']['index']
test.loc[bert_data['id'].tolist()]['text']

155285232084467712    I just hope Ian Bennett is first out of the se...
158551378271277058    Watched a movie yesterday #The70's on #OVTV an...
159200989268807680    I liked a @YouTube video http://t.co/dSPcjWDB ...
160323475427430400    A win for #LFC v Bolton will see them beat the...
162895027431948289    @DiscoMundial thanks! Drop an email to info@ka...
                                            ...                        
264248465440272384    When your lying in bed and your phone is on yo...
264252575879151616    Arsenal reportedly tracking the progress of Le...
264257216004120576    @LaFanMike7 I'd ask my roommate how he felt ab...
264258491529719808    The Spurs beat the Thunder 86-84 to win their ...
264264907137097728    @DrPhi1 Cuba. :) I'll tell you more after I bo...
Name: text, Length: 1624, dtype: object

In [10]:
orig_data

['I just hope Ian Bennett is first out of the selection machine tonight #terrierball #htafc #dontfancyArfieldingoal',
 "Watched a movie yesterday #The70's on #OVTV and was pleasantly surprised to see Michael Easton (John, #OLTL) in it.",
 'I liked a @YouTube video http://t.co/dSPcjWDB THSK - 4th Live Tour 27/31 - We are + Break up the she',
 "A win for #LFC v Bolton will see them beat their away points tally for the whole of last season. And we have to win tomorrow's game.",
 "@couriermail I may not agree with Wilkie's ideas but he has proven he is a man of principle...It's a pity Peter Garrett cannot say the same.",
 '@DiscoMundial thanks! Drop an email to info@kayafestival.co.uk regarding bookings. See you got Teengirl Fantasy tonight - amazing band!',
 'Not an Arse hater but happy to see them come back from 0-0 down last night v Bolton to win 0-0. Awesome showing with the Terry Henry too.',
 "There are still tickets available for the Mar 9 panel at Paleyfest. Let's make #Castle a so

In [5]:
bert_data

,id,pred_class,true_class,pred_prob,pred_entropy,raw_input,attribution_scores
0,155285232084467712,0,0,1.00,0.012701,[CLS] I just hope Ian Bennett is first out of ...,0.09 0.26 0.05 -0.05 -0.23 -0.09 0.09 0.29 0.1...
1,158551378271277058,0,0,1.00,0.015283,[CLS] Watch ##ed a movie yesterday # The ##70 ...,-0.27 0.09 -0.05 -0.06 0.33 0.38 -0.02 0.05 -0...
2,159200989268807680,0,0,1.00,0.008743,[CLS] I liked a @ YouTube video http : / / t ....,-0.19 -0.10 -0.63 0.31 -0.17 0.19 -0.24 0.21 -...
3,160323475427430400,0,0,0.73,0.584572,[CLS] A win for # L ##FC v Bolton will see the...,-0.21 0.06 0.29 -0.07 0.00 -0.02 0.11 -0.10 -0...
4,162895027431948289,0,0,1.00,0.005845,[CLS] @ Disco ##M ##und ##ial thanks ! Drop an...,-0.29 0.23 -0.18 -0.02 0.12 0.02 0.16 0.15 -0....
...,...,...,...,...,...,...,...
1619,264248465440272384,1,1,0.99,0.045548,[CLS] When your lying in bed and your phone is...,-0.14 0.11 0.07 0.18 -0.03 0.16 -0.04 0.02 0.0...
1620,264252575879151616,1,0,0.95,0.197404,[CLS] Arsenal reportedly tracking the progress...,0.18 -0.18 -0.41 0.14 -0.18 0.45 0.14 -0.06 0....
1621,264257216004120576,1,1,0.99,0.039263,[CLS] @ La ##F ##an ##M ##ike ##7 I ' d ask my...,0.02 0.07 -0.01 -0.02 -0.02 -0.03 -0.05 0.02 0...
1622,264258491529719808,0,0,0.99,0.058528,[CLS] The Spurs beat the Thunder 86 - 84 to wi...,-0.45 -0.09 0.22 0.02 0.06 0.03 0.01 0.15 -0.0...


In [61]:
if dataset == 'SST-2':
    text_label = "sentence"
    test_data = "Hummingbird"

else:
    text_label = "text"
    test_data = dataset

## Get Data

for k in ['GuidedBP', 'InputXGrad', 'IntegGrad', 'SmoothGrad']:
    bert_data = pd.read_csv(f"Saliency/Noise{noise_level}/RAW/{model}/{dataset}-{k}.tsv", sep="\t") # dataframe
    orig_data = load_from_disk(f"./Data/{test_data}")['test'][text_label] # text form
    

In [62]:

sentence = bert_data["raw_input"].iloc[0]
score = bert_data["attribution_scores"].iloc[0]
pred_probs = bert_data["pred_prob"].iloc[0]
orig_text = orig_data[0]

In [66]:
bert_tokens, attr_list = remove_cls_and_bert(sentence, score)
orig2bert, idx2token = match_bert_token_to_original(bert_tokens, orig_text, model)

In [67]:
orig2bert

{0: [0],
 1: [1, 2],
 2: [3],
 3: [4],
 4: [5],
 5: [6],
 6: [7],
 7: [8],
 8: [9],
 9: [10],
 10: [11],
 11: [12],
 12: [13, 14, 15],
 13: [16],
 14: [17]}

In [68]:

new_score = {}
for key, value in orig2bert.items():
    the_list = []
    for bert_idx in value:
        the_list.append(attr_list[bert_idx])
    np_list = np.array(the_list).astype(float)
    
    attr_avg = np.mean(np_list)
    new_score[key] = attr_avg


In [69]:
new_score

{0: 0.1,
 1: 0.14,
 2: 0.1,
 3: 0.21,
 4: 0.12,
 5: 0.11,
 6: 0.1,
 7: 0.11,
 8: 0.14,
 9: 0.11,
 10: 0.11,
 11: 0.08,
 12: 0.10666666666666667,
 13: 0.1,
 14: 0.07}

In [70]:
score

'0.55 0.10 0.13 0.15 0.10 0.21 0.12 0.11 0.10 0.11 0.14 0.11 0.11 0.08 0.15 0.07 0.10 0.10 0.07 0.17 0.13'

In [71]:
print(sentence)
print(orig_text)

#s an Ġart ful Ġ, Ġintelligent Ġfilm Ġthat Ġstays Ġwithin Ġthe Ġconfines Ġof Ġa Ġwell - established Ġgenre Ġ. Ġ #/s
an artful , intelligent film that stays within the confines of a well-established genre . 


In [72]:
orig2bert

{0: [0],
 1: [1, 2],
 2: [3],
 3: [4],
 4: [5],
 5: [6],
 6: [7],
 7: [8],
 8: [9],
 9: [10],
 10: [11],
 11: [12],
 12: [13, 14, 15],
 13: [16],
 14: [17]}

In [85]:
new_data_record[0]

{'pred_class': 1.0,
 'true_class': 1.0,
 'pred_prob': 1.0,
 'pred_entropy': 0.0022213368,
 'raw_input_ids': ['an',
  'artful',
  ',',
  'intelligent',
  'film',
  'that',
  'stays',
  'within',
  'the',
  'confines',
  'of',
  'a',
  'well-established',
  'genre',
  '.'],
 'word_attributions': [0.1,
  0.14,
  0.1,
  0.21,
  0.12,
  0.11,
  0.1,
  0.11,
  0.14,
  0.11,
  0.11,
  0.08,
  0.10666666666666667,
  0.1,
  0.07]}

In [86]:
visualize_text_pred(new_data_record, bert_data.id.to_list(), bert_data.pred_entropy.to_list(), f"test")

visualize prediction text to  test
done
finish writing to  test
